# 03.2 Say Hello to Pandas!

![hello pandas!](../img/panda.jpg)

The `Pandas` library is an immensely powerful tool for performing analysis on multidimensional (tabular) data in general, and time series data specifically. 

In [ ]:
from datetime import datetime
from datetime import timedelta
from dataretrieval import nwis
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

startDate = '1960-01-01'
endDate = '2024-12-31'

gage = '13185000' # Boise River Near Twin Springs

## 1. Datetime Objects in `Pandas`

Because it is often used for analyzing time series data, `Pandas` is equipped with many of the same capabilities to work with `datetime` objects and, indeed, extends these capabilities in useful ways. For example, although a bit weird, if we wanted to create a time series of dates with an interval of every 2 weeks, beginning on the first Tuesday after our start date above, we can do so: 

In [ ]:
tsDates_2W_Tues = pd.date_range(start=startDate, end=endDate, freq='2W-TUE')
tsDates_2W_Tues

Perhaps a bit more useful, but if we wanted to create a time series that started at a particular day and time, but had fifteen minute intervals, we could do something like the following: 

In [ ]:
tsDates_15min = pd.date_range(start='2025-10-01 00:00:00', end='2025-10-02 00:00:00', freq='15min')
tsDates_15min

As we can see, the `date_range` function can help us create a time series at almost any __regular__ interval we want, using the `freq` flag, which is referred to as the offset alias. The following documentation provides a table of different offset aliases: https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases and their descriptions. Can you create a time series of objects that have a monthly interval that starts on the first of the month between `startDate` and `endDate`, as defined above?

In [ ]:
tsDates = pd.date_range(start=startDate, end=endDate, freq='MS')
tsDates

If we wanted a day of the month that was anything other than the start (`freq='MS'`) or end of the month (`freq='ME'`), we can use the function `pd.DateOffset()` to add a specific number of days to each element of our time series. For example, in land modeling we have vegetation leaf area index (LAI) changing on a daily basis, but we only have monthly mean estimates of LAI from satellites. We, therefore, often assume that the value for the monthly mean is valid on the 15th of the month and then interpolate for each day by computing a weighted average of the two nearest months (i.e., the value of LAI on March 16 is the sum of the LAI on March 15 times (30/31) and the LAI on April 15times (1/31)). So, we might need a time series that is monthly in nature, but where the day of the month is 15, instead of 1:

In [ ]:
tsDates_midmonth = tsDates + pd.DateOffset(days=14)
tsDates_midmonth

Before going further, let's examine the specific data type associated with `tsDates`: 

In [ ]:
type(tsDates)

So we can see that this is a special kind of object that is like a standard Python `datetime` object, but is particular to `pandas` indexes. A bit further down, we'll see why this is important. In the mean time, we can use some built-in attributes to access individual elements of the `tsDates` like the month and year. 

In [ ]:
tsDates.month

In [ ]:
tsDates.year

If we look at the data type of `tsDates.year` we find that the datatype is still a `pandas` type object: 

In [ ]:
type(tsDates.year)

But, if we wanted to perform any mathematical operations on the years, we need to use the `.values` attribute to push the values of `tsDates.year` to a `numpy` array, like so:

In [ ]:
WY = tsDates.year.values

Verify that this is, indeed a numpy array: 

In [ ]:
type(WY)
WY.shape

### Challenge

Run the following code, examine the output and explain what it does:

In [ ]:
WY[tsDates.month > 9] = WY[tsDates.month > 9] + 1
WY

## 2. DataFrames in Pandas

Beyond __creating__ time series with Pandas, the primary way that we use Pandas is to deal with tabular data, of which time series data is a special case. For example, let's use the `dataretrieval` library again to get discharge data for the Boise River between our start and end dates. Specifically, let's look at what the `dataretrieval` library returns: 

In [ ]:
BoiseRiverQ = nwis.get_dv(sites=gage, parameterCd='00060', start=startDate, end=endDate)[0]
type(BoiseRiverQ)

Let's look at what is inside this `DataFrame`. What does the following look like, or remind you of? 

In [ ]:
BoiseRiverQ

In [ ]:
BoiseRiverQ.index

In [ ]:
BoiseRiverQ.plot(figsize=(12,8))

### `.groupby()`: The Most Powerful Operator In All of Python and Pandas? 

In the following code below, we use an operator on a `Pandas` dataframe to get the annual maximum value of daily discharge in the Boise River over the period from 1960 to 2024, as well as the unique values of the years. The `.groupby()` operator as used below, effectively tells `Pandas`: "group the column '00060_Mean' by the years in the index of the dataframe (the date), and then for each of those groups, give me the maximum." Ordinarily we would have resorted to writing `for` loops, looping over all years and then getting the maximum value within each year. No need to do so with `Pandas` and this actually reveals something pretty intriguing about modern computation and analysis of large datasets. 

In [ ]:
BoiseRiverQ['00060_Mean'].groupby(BoiseRiverQ.index.year)

In [ ]:
BoiseRiver_AnnMaxQ = BoiseRiverQ['00060_Mean'].groupby(BoiseRiverQ.index.year).max()
BoiseRiver_year = np.unique(BoiseRiverQ.index.year.values)

In [ ]:
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(6,4))
plt.plot(BoiseRiver_year, BoiseRiver_AnnMaxQ, 'kd')
plt.xlabel('Year')
plt.ylabel('Annual Maximum Flow [ft${}^3$/s]')
plt.grid('on')
plt.show()

### Challenge: Mean Monthly Flow

Can you use the `groupby()` function as show below, but instead, use it to compute and plot the mean monthly flow in the Boise River? 

In [ ]:
BoiseRiverQ['00060_Mean'].groupby([BoiseRiverQ.index.year,BoiseRiverQ.index.month]).mean().plot()